# MakeMore 
### Character-level language modeling
based on [A Neural Probabilistic Language Model](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)

processing and text generation operates at the level of individual characters.

In [4]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
words = open('../names.txt', 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [6]:
len(words)

32033

In [8]:
chars = sorted(list(set(''.join(words))))
s2i = {s:i+1 for i,s in enumerate(chars)}
s2i['.'] = 0

i2s = {i:s for s,i in s2i.items()}

print(i2s)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


## Building dataset

In [10]:
block_size = 3 # context lenght: how many characters do we take to predict the next one

X, Y = [], []

for w in words[:5]:

  print(w)

  context = [0] * block_size
  for ch in w + '.':
    idx = s2i[ch]
    X.append(context)
    Y.append(idx)
    print(''.join(i2s[i] for i in context), '--->', i2s[idx] )
    context = context[1:] + [idx] # shift the context by one and add the new character

X = torch.tensor(X)
Y = torch.tensor(Y)


emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [11]:
X.shape, Y.shape

(torch.Size([32, 3]), torch.Size([32]))

In [12]:
C = torch.randn((27,2)) # 27 characters, 2 dimensions

tensor([-1.6595, -0.2289])

In [13]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [14]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [19]:
torch.cat(torch.unbind(emb,1), dim=1)

tensor([[ 1.2433, -1.0059,  1.2433, -1.0059,  1.2433, -1.0059],
        [ 1.2433, -1.0059,  1.2433, -1.0059, -1.6595, -0.2289],
        [ 1.2433, -1.0059, -1.6595, -0.2289,  0.0682, -0.3035],
        [-1.6595, -0.2289,  0.0682, -0.3035,  0.0682, -0.3035],
        [ 0.0682, -0.3035,  0.0682, -0.3035, -0.5134, -1.8056],
        [ 1.2433, -1.0059,  1.2433, -1.0059,  1.2433, -1.0059],
        [ 1.2433, -1.0059,  1.2433, -1.0059, -0.1179,  0.2245],
        [ 1.2433, -1.0059, -0.1179,  0.2245, -0.0829,  0.1921],
        [-0.1179,  0.2245, -0.0829,  0.1921,  0.7625, -1.7553],
        [-0.0829,  0.1921,  0.7625, -1.7553, -1.5583,  1.2419],
        [ 0.7625, -1.7553, -1.5583,  1.2419,  0.7625, -1.7553],
        [-1.5583,  1.2419,  0.7625, -1.7553, -0.5134, -1.8056],
        [ 1.2433, -1.0059,  1.2433, -1.0059,  1.2433, -1.0059],
        [ 1.2433, -1.0059,  1.2433, -1.0059, -0.5134, -1.8056],
        [ 1.2433, -1.0059, -0.5134, -1.8056, -1.5583,  1.2419],
        [-0.5134, -1.8056, -1.5583,  1.2